# 因子计算与回测通用模板

本模板提供了从因子计算到策略回测的完整流程，适用于各种量化因子的开发和测试。

## 一、 模板功能

- **数据加载**: 统一的数据获取方法
- **因子计算**: 可扩展的因子计算框架
- **策略回测**: 完整的回测执行流程
- **性能分析**: 全面的结果评估
- **可视化**: 专业的图表展示

## 二、 使用步骤

1. 定义您的因子计算逻辑
2. 配置回测参数
3. 执行因子计算
4. 运行策略回测
5. 分析结果和可视化

## 1. 📦 导入必要模块

In [ ]:
# 基础库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from datetime import datetime, timedelta
import sys
import os

# 添加项目路径
sys.path.append('../data_manager')
sys.path.append('../backtest_engine')
sys.path.append('../factor_library/fundamental')

# 导入项目模块
from data import DataManager
from engine import BacktestEngine, run_backtest
from performance import PerformanceAnalyzer

# 设置
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 模块导入完成!")

## 2. ⚙️ 配置参数

设置因子计算和回测的基本参数。

In [ ]:
# =========================
# 📊 基础配置参数
# =========================

# 时间范围
START_DATE = '2024-01-01'
END_DATE = '2024-02-29'

# 股票池（可根据需要修改）
STOCK_UNIVERSE = [
    '000001.SZ', '000002.SZ', '000858.SZ',
    '600000.SH', '600036.SH', '600519.SH'
]

# 回测配置
BACKTEST_CONFIG = {
    'rebalance_freq': 'weekly',    # 调仓频率: daily, weekly, monthly
    'transaction_cost': 0.0        # 交易费用: 0.0 表示零手续费
}

print("⚙️ 参数配置完成!")
print(f"  时间范围: {START_DATE} 至 {END_DATE}")
print(f"  股票数量: {len(STOCK_UNIVERSE)}")
print(f"  调仓频率: {BACKTEST_CONFIG['rebalance_freq']}")
print(f"  交易费用: {BACKTEST_CONFIG['transaction_cost']}")

## 3. 🔧 初始化数据管理器

创建数据管理器实例，用于后续的数据获取。

In [ ]:
# 初始化数据管理器
data_manager = DataManager()

print("🔧 数据管理器初始化完成!")
print(f"  数据路径: {data_manager.clean_data_path}")
print(f"  缓存状态: {'启用' if hasattr(data_manager, '_cache') else '禁用'}")

## 4. 🧮 定义因子计算函数

这里提供一个通用的因子计算框架，您可以根据需要修改具体的计算逻辑。

In [ ]:
def calculate_size_factor(data_manager, start_date, end_date, stock_codes=None):
    """
    计算规模因子（示例）
    
    参数:
        data_manager: 数据管理器实例
        start_date: 开始日期
        end_date: 结束日期  
        stock_codes: 股票代码列表，None表示全部
    
    返回:
        DataFrame: MultiIndex (trade_date, stock_code) 格式的因子数据
    """
    print("🧮 开始计算规模因子...")
    
    # 获取日行情数据
    daily_data = data_manager.get_stock_daily_data(
        start_date=start_date,
        end_date=end_date,
        stock_codes=stock_codes
    )
    
    # 获取基础信息数据
    basic_data = data_manager.get_stock_basic_data()
    
    # 合并数据以获取总股本信息
    merged_data = daily_data.merge(
        basic_data[['ts_code', 'total_share']],
        left_on='stock_code',
        right_on='ts_code',
        how='left'
    )
    
    # 计算市值 = 收盘价 × 总股本 / 10000（单位：万元）
    merged_data['market_cap'] = merged_data['close'] * merged_data['total_share'] / 10000
    
    # 计算对数市值因子
    merged_data['log_market_cap'] = np.log(merged_data['market_cap'])
    
    # 转换为MultiIndex格式
    factor_data = merged_data.set_index(['trade_date', 'stock_code'])[['log_market_cap']]
    
    # 重命名列
    factor_data.columns = ['factor']
    
    print(f"✅ 规模因子计算完成!")
    print(f"  数据形状: {factor_data.shape}")
    print(f"  因子均值: {factor_data['factor'].mean():.4f}")
    print(f"  因子标准差: {factor_data['factor'].std():.4f}")
    
    return factor_data


def calculate_custom_factor(data_manager, start_date, end_date, stock_codes=None):
    """
    自定义因子计算模板（请根据需要修改）
    
    这里可以实现您自己的因子计算逻辑
    """
    print("🧮 开始计算自定义因子...")
    
    # TODO: 在这里实现您的因子计算逻辑
    # 示例：简单的价格因子
    daily_data = data_manager.get_stock_daily_data(
        start_date=start_date,
        end_date=end_date,
        stock_codes=stock_codes
    )
    
    # 计算5日收益率作为示例因子
    daily_data = daily_data.sort_values(['stock_code', 'trade_date'])
    daily_data['return_5d'] = daily_data.groupby('stock_code')['close'].pct_change(5)
    
    # 转换为MultiIndex格式
    factor_data = daily_data.set_index(['trade_date', 'stock_code'])[['return_5d']]
    factor_data.columns = ['factor']
    
    # 移除空值
    factor_data = factor_data.dropna()
    
    print(f"✅ 自定义因子计算完成!")
    print(f"  数据形状: {factor_data.shape}")
    
    return factor_data

print("✅ 因子计算函数定义完成!")

## 5. 🚀 执行因子计算

运行因子计算并检查结果。

In [ ]:
# 选择要使用的因子计算方法
FACTOR_METHOD = 'size'  # 可选: 'size', 'custom'

print(f"🚀 开始计算因子 ({FACTOR_METHOD})...")

if FACTOR_METHOD == 'size':
    factor_data = calculate_size_factor(
        data_manager=data_manager,
        start_date=START_DATE,
        end_date=END_DATE,
        stock_codes=STOCK_UNIVERSE
    )
elif FACTOR_METHOD == 'custom':
    factor_data = calculate_custom_factor(
        data_manager=data_manager,
        start_date=START_DATE,
        end_date=END_DATE,
        stock_codes=STOCK_UNIVERSE
    )
else:
    raise ValueError(f"不支持的因子方法: {FACTOR_METHOD}")

# 因子数据基本信息
print(f"\n📊 因子数据概览:")
print(f"  时间范围: {factor_data.index.get_level_values('trade_date').min()} 至 {factor_data.index.get_level_values('trade_date').max()}")
print(f"  股票数量: {factor_data.index.get_level_values('stock_code').nunique()}")
print(f"  观测数量: {len(factor_data)}")
print(f"  缺失值: {factor_data.isnull().sum().sum()}")

# 展示前几行数据
print(f"\n📋 因子数据预览:")
print(factor_data.head(10))

# 因子统计信息
print(f"\n📈 因子统计信息:")
print(factor_data.describe())

## 6. 🎯 执行策略回测

使用计算好的因子数据运行回测。

In [ ]:
print("🎯 开始执行策略回测...")

try:
    # 执行回测
    portfolio_returns, positions = run_backtest(
        factor_data=factor_data,
        data_manager=data_manager,
        start_date=START_DATE,
        end_date=END_DATE,
        **BACKTEST_CONFIG
    )
    
    print("✅ 回测执行完成!")
    print(f"  组合收益序列长度: {len(portfolio_returns)}")
    print(f"  调仓次数: {len(positions)}")
    
    # 基本业绩指标
    total_return = (portfolio_returns + 1).prod() - 1
    annualized_return = (1 + total_return) ** (252 / len(portfolio_returns)) - 1
    volatility = portfolio_returns.std() * np.sqrt(252)
    sharpe_ratio = annualized_return / volatility if volatility > 0 else 0
    max_drawdown = (portfolio_returns.cumsum().expanding().max() - portfolio_returns.cumsum()).max()
    
    print(f"\n📊 基本业绩指标:")
    print(f"  总收益率: {total_return:.4f} ({total_return:.2%})")
    print(f"  年化收益率: {annualized_return:.4f} ({annualized_return:.2%})")
    print(f"  年化波动率: {volatility:.4f} ({volatility:.2%})")
    print(f"  夏普比率: {sharpe_ratio:.4f}")
    print(f"  最大回撤: {max_drawdown:.4f} ({max_drawdown:.2%})")
    
    # 检查调仓情况
    if len(positions) > 0:
        print(f"\n🔄 调仓情况:")
        print(f"  首次调仓: {positions.iloc[0]['date']}")
        print(f"  最后调仓: {positions.iloc[-1]['date']}")
        print(f"  平均持仓数: {np.mean([len(p['positions']) for p in positions['positions']]):.1f}")
        
except Exception as e:
    print(f"❌ 回测执行失败: {e}")
    print("💡 请检查因子数据格式和参数配置")
    raise

## 7. 📈 详细性能分析

使用性能分析器进行深入分析。

In [ ]:
print("📈 开始详细性能分析...")

try:
    # 创建性能分析器
    analyzer = PerformanceAnalyzer()
    
    # 执行完整分析
    analysis_results = analyzer.analyze_performance(
        portfolio_returns=portfolio_returns,
        factor_data=factor_data
    )
    
    print("\n✅ 详细性能分析完成!")
    
    # 显示性能指标
    if 'performance_metrics' in analysis_results:
        print("\n📊 详细业绩指标:")
        metrics = analysis_results['performance_metrics']
        
        for key, value in metrics.items():
            if isinstance(value, (int, float)):
                if 'ratio' in key.lower() or 'return' in key.lower():
                    print(f"  {key}: {value:.4f}")
                else:
                    print(f"  {key}: {value:.4f}")
            else:
                print(f"  {key}: {value}")
    
    # 显示IC分析结果
    if 'ic_analysis' in analysis_results:
        print("\n🎯 IC分析结果:")
        ic_metrics = analysis_results['ic_analysis']
        
        for key, value in ic_metrics.items():
            if isinstance(value, (int, float)):
                print(f"  {key}: {value:.4f}")
            else:
                print(f"  {key}: {value}")
    
    # 保存分析结果
    analysis_summary = {
        'factor_method': FACTOR_METHOD,
        'backtest_config': BACKTEST_CONFIG,
        'time_range': f"{START_DATE} 至 {END_DATE}",
        'stock_count': len(STOCK_UNIVERSE),
        'analysis_results': analysis_results
    }
    
except Exception as e:
    print(f"⚠️ 详细分析失败: {e}")
    print("💡 将使用基础指标作为备选")
    
    analysis_summary = {
        'factor_method': FACTOR_METHOD,
        'backtest_config': BACKTEST_CONFIG,
        'basic_metrics': {
            'total_return': total_return,
            'annualized_return': annualized_return,
            'volatility': volatility,
            'sharpe_ratio': sharpe_ratio,
            'max_drawdown': max_drawdown
        }
    }

## 8. 📊 结果可视化

创建专业的图表展示回测结果。

In [ ]:
print("📊 开始创建可视化图表...")

# 计算累积收益
cumulative_returns = (portfolio_returns + 1).cumprod()

# 创建综合图表
fig = plt.figure(figsize=(15, 12))

# 1. 累积收益曲线
ax1 = plt.subplot(3, 2, 1)
ax1.plot(cumulative_returns.index, cumulative_returns.values, 
         linewidth=2, color='blue', label=f'{FACTOR_METHOD.upper()}因子策略')
ax1.set_title('📈 累积收益曲线', fontsize=12, fontweight='bold')
ax1.set_ylabel('累积收益倍数')
ax1.grid(True, alpha=0.3)
ax1.legend()

# 添加收益统计
final_return = cumulative_returns.iloc[-1] - 1
ax1.text(0.02, 0.98, f'总收益: {final_return:.2%}', 
         transform=ax1.transAxes, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))

# 2. 日收益率分布
ax2 = plt.subplot(3, 2, 2)
ax2.hist(portfolio_returns, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
ax2.set_title('📊 日收益率分布', fontsize=12, fontweight='bold')
ax2.set_xlabel('日收益率')
ax2.set_ylabel('频次')
ax2.grid(True, alpha=0.3)

# 添加统计线
mean_return = portfolio_returns.mean()
ax2.axvline(mean_return, color='red', linestyle='--', alpha=0.8, label=f'均值: {mean_return:.4f}')
ax2.legend()

# 3. 回撤分析
ax3 = plt.subplot(3, 2, 3)
cumulative_max = cumulative_returns.expanding().max()
drawdown = (cumulative_returns - cumulative_max) / cumulative_max
ax3.fill_between(drawdown.index, drawdown.values, 0, color='red', alpha=0.3)
ax3.plot(drawdown.index, drawdown.values, color='red', linewidth=1)
ax3.set_title('📉 回撤分析', fontsize=12, fontweight='bold')
ax3.set_ylabel('回撤比例')
ax3.grid(True, alpha=0.3)

# 4. 因子分布
ax4 = plt.subplot(3, 2, 4)
factor_values = factor_data['factor'].dropna()
ax4.hist(factor_values, bins=30, alpha=0.7, color='green', edgecolor='black')
ax4.set_title('🧮 因子值分布', fontsize=12, fontweight='bold')
ax4.set_xlabel('因子值')
ax4.set_ylabel('频次')
ax4.grid(True, alpha=0.3)

# 5. 滚动收益率
ax5 = plt.subplot(3, 2, 5)
rolling_returns = portfolio_returns.rolling(window=20).apply(lambda x: (1 + x).prod() - 1)
ax5.plot(rolling_returns.index, rolling_returns.values, linewidth=1, color='orange')
ax5.set_title('📈 20日滚动收益率', fontsize=12, fontweight='bold')
ax5.set_ylabel('滚动收益率')
ax5.grid(True, alpha=0.3)

# 6. 业绩指标总结
ax6 = plt.subplot(3, 2, 6)
ax6.axis('off')

# 创建业绩指标文本
metrics_text = f"""
📊 核心业绩指标

总收益率: {total_return:.2%}
年化收益率: {annualized_return:.2%}
年化波动率: {volatility:.2%}
夏普比率: {sharpe_ratio:.3f}
最大回撤: {max_drawdown:.2%}

🔄 回测配置

因子类型: {FACTOR_METHOD.upper()}
调仓频率: {BACKTEST_CONFIG['rebalance_freq']}
交易费用: {BACKTEST_CONFIG['transaction_cost']:.1%}
调仓次数: {len(positions)}
股票数量: {len(STOCK_UNIVERSE)}
"""

ax6.text(0.1, 0.9, metrics_text, transform=ax6.transAxes, 
         fontsize=10, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

plt.tight_layout()
plt.show()

print("✅ 可视化图表创建完成!")

## 9. 🔍 因子有效性分析

深入分析因子的有效性和稳定性。

In [ ]:
print("🔍 开始因子有效性分析...")

# 按时间分析因子表现
def analyze_factor_by_time(factor_data, portfolio_returns):
    """
    按时间维度分析因子表现
    """
    # 因子时间序列统计
    factor_by_date = factor_data.groupby('trade_date')['factor'].agg([
        'mean', 'std', 'min', 'max', 'count'
    ]).round(4)
    
    print("📅 因子时间序列统计:")
    print(factor_by_date.head())
    
    # 月度收益分解
    monthly_returns = portfolio_returns.resample('M').apply(lambda x: (1 + x).prod() - 1)
    
    print(f"\n📆 月度收益分解:")
    for date, ret in monthly_returns.items():
        print(f"  {date.strftime('%Y-%m')}: {ret:.4f} ({ret:.2%})")
    
    return factor_by_date, monthly_returns

# 因子分位数分析
def analyze_factor_quantiles(factor_data, n_quantiles=5):
    """
    分析因子分位数分布
    """
    print(f"\n📊 因子 {n_quantiles} 分位数分析:")
    
    # 计算分位数
    factor_values = factor_data['factor'].dropna()
    quantiles = np.linspace(0, 1, n_quantiles + 1)
    quantile_values = factor_values.quantile(quantiles)
    
    for i, (q, val) in enumerate(quantile_values.items()):
        print(f"  {q:.0%} 分位数: {val:.4f}")
    
    # 分位数内股票数量分布
    factor_data_copy = factor_data.copy()
    factor_data_copy['quantile'] = pd.qcut(factor_data_copy['factor'], 
                                          q=n_quantiles, labels=False, duplicates='drop')
    
    quantile_counts = factor_data_copy.groupby('quantile').size()
    print(f"\n📈 各分位数股票数量:")
    for q, count in quantile_counts.items():
        print(f"  分位数 {q+1}: {count} 只股票")
    
    return quantile_values, quantile_counts

# 执行分析
try:
    factor_time_stats, monthly_rets = analyze_factor_by_time(factor_data, portfolio_returns)
    quantile_vals, quantile_counts = analyze_factor_quantiles(factor_data)
    
    print("\n✅ 因子有效性分析完成!")
    
except Exception as e:
    print(f"⚠️ 因子分析出现问题: {e}")
    print("💡 继续使用基础分析结果")

## 10. 📋 结果总结与报告

生成完整的分析报告。

In [ ]:
print("📋 生成分析报告...")

# 创建完整报告
report = f"""
{'='*60}
               因子策略回测分析报告
{'='*60}

🎯 基本信息
  因子类型: {FACTOR_METHOD.upper()}因子
  回测期间: {START_DATE} 至 {END_DATE}
  股票数量: {len(STOCK_UNIVERSE)} 只
  调仓频率: {BACKTEST_CONFIG['rebalance_freq']}
  交易费用: {BACKTEST_CONFIG['transaction_cost']:.1%}
  调仓次数: {len(positions)}

📊 核心业绩指标
  总收益率: {total_return:.4f} ({total_return:.2%})
  年化收益率: {annualized_return:.4f} ({annualized_return:.2%})
  年化波动率: {volatility:.4f} ({volatility:.2%})
  夏普比率: {sharpe_ratio:.4f}
  最大回撤: {max_drawdown:.4f} ({max_drawdown:.2%})

🧮 因子统计信息
  观测数量: {len(factor_data)}
  因子均值: {factor_data['factor'].mean():.4f}
  因子标准差: {factor_data['factor'].std():.4f}
  因子最小值: {factor_data['factor'].min():.4f}
  因子最大值: {factor_data['factor'].max():.4f}
  缺失值数量: {factor_data.isnull().sum().sum()}

🎯 策略表现评价
"""

# 添加策略表现评价
if sharpe_ratio > 1.0:
    performance_rating = "🌟 优秀"
elif sharpe_ratio > 0.5:
    performance_rating = "👍 良好"
elif sharpe_ratio > 0.0:
    performance_rating = "⚠️ 一般"
else:
    performance_rating = "❌ 较差"

report += f"  整体表现: {performance_rating} (夏普比率: {sharpe_ratio:.3f})\n"

if max_drawdown < 0.05:
    risk_rating = "🛡️ 低风险"
elif max_drawdown < 0.10:
    risk_rating = "⚖️ 中等风险"
else:
    risk_rating = "⚠️ 高风险"

report += f"  风险水平: {risk_rating} (最大回撤: {max_drawdown:.2%})\n"

report += f"""
💡 优化建议
  1. 如果夏普比率较低，考虑优化因子计算方法
  2. 如果最大回撤过大，考虑加入风险控制措施
  3. 测试不同的调仓频率以优化交易成本
  4. 扩大股票池以提高策略稳定性
  5. 结合其他因子构建多因子模型

📅 报告生成时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
{'='*60}
"""

print(report)

# 保存报告到变量供后续使用
final_report = {
    'report_text': report,
    'factor_data': factor_data,
    'portfolio_returns': portfolio_returns,
    'positions': positions,
    'analysis_summary': analysis_summary,
    'config': {
        'factor_method': FACTOR_METHOD,
        'start_date': START_DATE,
        'end_date': END_DATE,
        'stock_universe': STOCK_UNIVERSE,
        'backtest_config': BACKTEST_CONFIG
    }
}

print("\n✅ 分析报告生成完成!")
print("📊 所有结果已保存在 final_report 变量中")

## 11. 🚀 下一步行动

### 🎯 模板使用指南

1. **修改因子计算逻辑**: 在第4节中的 `calculate_custom_factor` 函数中实现您的因子计算逻辑
2. **调整回测参数**: 修改第2节中的配置参数以适应您的需求
3. **扩展股票池**: 根据研究需要调整 `STOCK_UNIVERSE` 列表
4. **优化策略**: 基于分析结果调整因子计算方法或回测参数

### 🔧 常用修改位置

- **时间范围**: 修改 `START_DATE` 和 `END_DATE`
- **股票池**: 修改 `STOCK_UNIVERSE` 列表
- **调仓频率**: 修改 `BACKTEST_CONFIG['rebalance_freq']`
- **交易费用**: 修改 `BACKTEST_CONFIG['transaction_cost']`
- **因子计算**: 修改 `calculate_custom_factor` 函数

### 📚 进阶功能

- 多因子组合策略
- 行业中性化处理
- 风险模型集成
- 实盘交易接口

---

**🎉 恭喜！您已完成因子策略的完整开发流程！**

此模板提供了从因子计算到策略回测的完整框架，您可以根据具体需求进行定制和扩展。